In [162]:
#Syntax to download data through terminal: rsync -aP kkulbay@student-shell.sys.kth.se:/afs/kth.se/misc/csc/dept/tmh/corpora/tidigits/disc_4.2.1/tidigits/test/ ~/Desktop/KTH/DT2119_Speech_and_Speaker_Recognition/dt2119/DT2119_Speech/LAB3/tidigits/test

import numpy as np
import lab3_toolsSPYDER as tools
import lab3_proto as proto
import lab1_proto as proto1
import lab1_tools as tools1
import lab2_proto as proto2
import lab2_tools as tools2
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler
#import tensorflow as tf

# 4.1 Target Class Definition

In [163]:
phoneHMMs = np.load('lab2_models_all.npz',allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs.keys())
nstates = {phone: phoneHMMs[phone]['means'].shape[0] for phone in phones}
# stateList = [ph + '_' + str(id) for ph in phones for id in range(nstates[ph])]

# np.save("stateList.npy",stateList)

#loading genereated list from above 
stateList = np.load("stateList.npy", allow_pickle=True).tolist()
# print(stateList)
# print(phoneHMMs.keys())

# 4.2 Forced Alignment

In [164]:

#lmfccEXAMPLE = np.load("lmfccEXAMPLE.npy", allow_pickle=True)

filename = 'z43a.wav'
samples, samplingrate = tools.loadAudio(filename)

lmfccEXAMPLE = proto1.mfcc(samples,samplingrate=samplingrate)


# plt.pcolormesh(lmfccEXAMPLE)
# plt.title("MFCCs test")
# plt.show()

wordTrans = list(tools.path2info(filename)[2])
assert wordTrans == ['z', '4', '3']
#-------
prondict = {} 
prondict['o'] = ['ow']
prondict['z'] = ['z', 'iy', 'r', 'ow']
prondict['1'] = ['w', 'ah', 'n']
prondict['2'] = ['t', 'uw']
prondict['3'] = ['th', 'r', 'iy']
prondict['4'] = ['f', 'ao', 'r']
prondict['5'] = ['f', 'ay', 'v']
prondict['6'] = ['s', 'ih', 'k', 's']
prondict['7'] = ['s', 'eh', 'v', 'ah', 'n']
prondict['8'] = ['ey', 't']
prondict['9'] = ['n', 'ay', 'n']

phoneTrans = proto.words2phones(wordTrans, prondict)
assert phoneTrans == ['sil', 'z', 'iy', 'r', 'ow', 'sp', 'f', 'ao', 'r', 'sp', 'th', 'r', 'iy', 'sp', 'sil']
#---------
utteranceHMM = proto2.concatHMMs(phoneHMMs, phoneTrans)
stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans
                  for stateid in range(nstates[phone])]
# print(stateTrans)
# print(stateList)
# print(utteranceHMM.keys())
# utteranceHMM["name"]

#-----
lab3example = np.load("lab3_example.npz", allow_pickle=True)

#print(utteranceHMM)
#print(lab3example["example"])#.f.filename)


In [165]:
obsloglik = tools2.log_multivariate_normal_density_diag(lmfccEXAMPLE,utteranceHMM["means"],utteranceHMM["covars"])
vloglik, vpath = proto2.viterbi(obsloglik,np.log(utteranceHMM["startprob"]),np.log(utteranceHMM["transmat"][:-1,:-1]))

# print(vpath)
# print(len(vpath),len(stateTrans))
viterbiStateTrans = []
for i,stateID in enumerate(vpath):
    viterbiStateTrans.append(stateTrans[stateID])

CTRLviterbiStateTrans = ['sil_0', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_2', 'z_0', 'z_0', 'z_0', 'z_0', 'z_1', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_1', 'iy_2', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_1', 'r_2', 'ow_0', 'ow_1', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'f_0', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_2', 'ao_0', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'r_0', 'r_0', 'r_0', 'r_1', 'r_2', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_1', 'th_1', 'th_1', 'th_2', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_1', 'r_2', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_1', 'iy_1', 'iy_2', 'iy_2', 'iy_2', 'iy_2', 'iy_2', 'iy_2', 'iy_2', 'iy_2', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_1', 'sil_2']

assert CTRLviterbiStateTrans == viterbiStateTrans # This assertion will be incorrect since the loaded example is based on double liftering, main data is not
#tools.frames2trans(viterbiStateTrans, outfilename='z43a.lab')

/var/folders/q9/fn0bq0g17g1_dkmkvbvcxll00000gn/T/ipykernel_26829/720878886.py:2: RuntimeWarning: divide by zero encountered in log
  vloglik, vpath = proto2.viterbi(obsloglik,np.log(utteranceHMM["startprob"]),np.log(utteranceHMM["transmat"][:-1,:-1]))


# 4.3 Feature Extraction

In [166]:
# traindata = proto.extractDATASET("tidigits/train")
# testdata = proto.extractDATASET("tidigits/test")

# np.save("traindata.npy",traindata)
# np.save("testdata.npy",testdata)

traindata = np.load("traindata.npy", allow_pickle=True)
testdata = np.load("testdata.npy", allow_pickle=True)

# z43aINDEX = None
# for i,datapt in enumerate(traindata):
#     if datapt["filename"] == "tidigits/train/man/nw/z43a.wav":
#         z43aINDEX = i

z43aINDEX = 3374
assert proto._stateIDs2stateNames(traindata[z43aINDEX]["targets"],stateList) == CTRLviterbiStateTrans
print(traindata[4000]["filename"])



tidigits/train/man/ff/o3oo5a.wav


# 4.4 Training and Validation Sets

In [167]:
#head, tail = os.path.split(traindata[4000]["filename"])
print(traindata[4000]["filename"])
#print(head, tail)

#print(proto._getGenderIdFile(traindata[4000]["filename"]))
#print(proto._getNspeakerNgender(traindata))

Nspeakers, Nmen, Nwomen = proto._getNspeakerNgender(traindata)
print("Nspeakers:",Nspeakers, "Nmen:",Nmen, "Nwomen:",Nwomen )



tidigits/train/man/ff/o3oo5a.wav
Nspeakers: 112 Nmen: 4235 Nwomen: 4388


As can be seen above, we seem to have a 50/50 distribution of men/women. So if we take 10% and 90% of each gender pool, we should still have the same distribution in each subset.

In [168]:
traindataMen = proto.getSubset(traindata,"man")
traindataWomen = proto.getSubset(traindata,"woman")

NspeakersMen= proto._getNspeakerNgender(traindataMen)[0]
NspeakersWomen= proto._getNspeakerNgender(traindataWomen)[0]

print("NspeakersMen:",NspeakersMen, "NspeakersWomen:",NspeakersWomen)



NspeakersMen: 55 NspeakersWomen: 57


What we now do is that we seperate each gender subset into dictionaries where each key in the dictionary is a speaker ID. This key holds arrays with datapoints only belonging to that speaker. Since we want a 10% subset we calculate how many datapts are needed from each gender. Then these datapts are taken to that approximate value from these unique speakers. Meaning that whatever datapts are remaining only belong to speakers not in the 10% susbet. 

In [169]:
# traindataMen90, traindataMen10 = proto.splitDatasetUnique(traindataMen,0.9)
# traindataWomen90, traindataWomen10 = proto.splitDatasetUnique(traindataWomen,0.9)

speakerUniqueSubsetsMen = proto.splitSpeakerSubsets(traindataMen)
speakerUniqueSubsetsWomen = proto.splitSpeakerSubsets(traindataWomen)

N10Men = int(0.1*len(traindataMen))
N10Women = int(0.1*len(traindataWomen))

print("For the 10% subset we need about",N10Men, "men samples and ", N10Women, "women samples")

# print(speakerUniqueSubsetsMen.keys())
# print(len(speakerUniqueSubsetsMen["pd"])+len(speakerUniqueSubsetsMen["sj"])+len(speakerUniqueSubsetsMen["pb"])+len(speakerUniqueSubsetsMen["dn"])+len(speakerUniqueSubsetsMen["kd"])+len(speakerUniqueSubsetsMen["nr"]))
# print(speakerUniqueSubsetsWomen.keys())
# print(len(speakerUniqueSubsetsWomen["pm"])+len(speakerUniqueSubsetsWomen["sp"])+len(speakerUniqueSubsetsWomen["pk"])+len(speakerUniqueSubsetsWomen["pe"])+len(speakerUniqueSubsetsWomen["pp"])+len(speakerUniqueSubsetsWomen["ms"]))

speakerListMen10 = ["pd","sj","pb","dn","kd","nr"]
speakerListWomen10 = ["pm","sp","pk","pe","pp","ms"]

traindataMen90 = []
traindataMen10 = []

for datapt in traindataMen:
    filename = datapt["filename"]
    speakerID = proto._getGenderIdFile(filename)[1]
    if speakerID in speakerListMen10:
        traindataMen10.append(datapt)
    else:
        traindataMen90.append(datapt)

traindataWomen90 = []
traindataWomen10 = []

for datapt in traindataWomen:
    filename = datapt["filename"]
    speakerID = proto._getGenderIdFile(filename)[1]
    if speakerID in speakerListWomen10:
        traindataWomen10.append(datapt)
    else:
        traindataWomen90.append(datapt)

# print(len(traindataMen90), len(traindataMen10))
# print(len(traindataWomen90), len(traindataWomen10))



For the 10% subset we need about 423 men samples and  438 women samples


We now combine the seperate gender subsets into two subsets of 90/10 division

In [170]:
traindata90 = traindataMen90 + traindataWomen90
traindata10 = traindataMen10 + traindataWomen10

print("Ntraindata90:",len(traindata90),"Ntraindata10:",len(traindata10))
print("Ntraindata:",len(traindata))

Ntraindata90: 7699 Ntraindata10: 924
Ntraindata: 8623


# 4.5 Acoustic Context (Dynamic Features)

In [171]:
M = traindata90[3]["lmfcc"]
Mstack = proto.stackMatrix(M)

print(M.shape,Mstack.shape)


(127, 13) (127, 91)


The boundary cases were simply handled with zero vectors for the feature vectors, since they're not so importaant as each utterance starts and ends with silence. 

In [172]:
# proto.stackDataset(traindata90)
# proto.stackDataset(traindata10)
# proto.stackDataset(testdata)

# np.save("traindata90Stacked.npy",traindata90)
# np.save("traindata10Stacked.npy",traindata10)
# np.save("testdataStacked.npy",testdata)

# traindata90 = np.load("traindata90Stacked.npy", allow_pickle=True)
# traindata10 = np.load("traindata10Stacked.npy", allow_pickle=True)
# testdata = np.load("testdataStacked.npy", allow_pickle=True)


# 4.6 Feature Standardisation $\color{red}{!!!}$

Think about the implications of these different strategies. In the third case, what will happen with the very short utterances in the files containing isolated digits? 

In [173]:
# data = traindata90[3]["lmfcc"]
# scaler = StandardScaler().fit(data)
# dataScaled = scaler.transform(data)
# print(dataScaled.mean(axis=0))
# print(dataScaled.std(axis=0))

# proto.standardiseEachUtterance(traindata90)
# proto.standardiseEachUtterance(traindata10)
# proto.standardiseEachUtterance(testdata)

# print("on traindata90")
# np.save("traindata90Stand.npy",traindata90)
# print("on traindata10")
# np.save("traindata10Stand.npy",traindata10)
# print("on test")
# np.save("testdataStand.npy",testdata)

#traindata90 = np.load("traindata90Stand.npy", allow_pickle=True)
# traindata10 = np.load("traindata10Stand.npy", allow_pickle=True)
#testdata = np.load("testdataStand.npy", allow_pickle=True)

# print(traindata90[5]["lmfcc"].mean(axis=0),traindata90[5]["lmfcc"].std(axis=0))
# print(traindata10[34]["lmfcc"].mean(axis=0),traindata10[34]["lmfcc"].std(axis=0))
# print(testdata[2000]["lmfcc"].mean(axis=0),testdata[2000]["lmfcc"].std(axis=0))
# print("OK, means are approx 0 and variance is 1 for each utterance")



## Flatten data

In [174]:
#lmfcc_train_x,mspec_train_x,dlmfcc_train_x,dmspec_train_x,train_y = proto.flattenData(traindata90)
#np.savez("flatDataTrain.npz", lmfcc_train_x=lmfcc_train_x,mspec_train_x=mspec_train_x,dlmfcc_train_x=dlmfcc_train_x,dmspec_train_x=dmspec_train_x,train_y=train_y)
#lmfcc_val_x,mspec_val_x,dlmfcc_val_x,dmspec_val_x,val_y = proto.flattenData(traindata10)
#lmfcc_test_x,mspec_test_x,dlmfcc_test_x,dmspec_test_x,test_y = proto.flattenData(testdata)

#np.savez("flatDataVal.npz", lmfcc_val_x=lmfcc_val_x,mspec_val_x=mspec_val_x,dlmfcc_val_x=dlmfcc_val_x,dmspec_val_x=dmspec_val_x,val_y=val_y)
#np.savez("flatDataTest.npz", lmfcc_test_x=lmfcc_test_x,mspec_test_x=mspec_test_x,dlmfcc_test_x=dlmfcc_test_x,dmspec_test_x=dmspec_test_x,test_y=test_y)

#----
trainFile = np.load("flatDataTrain.npz")
lmfcc_train_x=trainFile["lmfcc_train_x"]
mspec_train_x=trainFile["mspec_train_x"]
dlmfcc_train_x=trainFile["dlmfcc_train_x"]
dmspec_train_x=trainFile["dmspec_train_x"]
train_y=trainFile["train_y"]
print(lmfcc_train_x.shape,mspec_train_x.shape,dlmfcc_train_x.shape,dmspec_train_x.shape,train_y.shape)
#----
valFile = np.load("flatDataVal.npz")
lmfcc_val_x=valFile["lmfcc_val_x"]
mspec_val_x=valFile["mspec_val_x"]
dlmfcc_val_x=valFile["dlmfcc_val_x"]
dmspec_val_x=valFile["dmspec_val_x"]
val_y=valFile["val_y"]
print(lmfcc_val_x.shape,mspec_val_x.shape,dlmfcc_val_x.shape,dmspec_val_x.shape,val_y.shape)
#----
testFile = np.load("flatDataTest.npz")
lmfcc_test_x=testFile["lmfcc_test_x"]
mspec_test_x=testFile["mspec_test_x"]
dlmfcc_test_x=testFile["dlmfcc_test_x"]
dmspec_test_x=testFile["dmspec_test_x"]
test_y=testFile["test_y"]
print(lmfcc_test_x.shape,mspec_test_x.shape,dlmfcc_test_x.shape,dmspec_test_x.shape,test_y.shape)

Nstring = 0
for i,el in enumerate(train_y):
    try: 
        int(el)
    except ValueError:
        Nstring += 1
        print(el)
        train_y[i] = stateList.index(el)
print(Nstring)

#np.savez("flatDataTrain.npz", lmfcc_train_x=lmfcc_train_x,mspec_train_x=mspec_train_x,dlmfcc_train_x=dlmfcc_train_x,dmspec_train_x=dmspec_train_x,train_y=train_y)

(1347549, 13) (1347549, 40) (1347549, 91) (1347549, 280) (1347549,)
(159843, 13) (159843, 40) (159843, 91) (159843, 280) (159843,)
(1527014, 13) (1527014, 40) (1527014, 91) (1527014, 280) (1527014,)
0
